In [14]:
import pandas as pd
import numpy as np
from six.moves import cPickle as pickle

In [2]:
data = pd.read_csv('UNSW_NB15_testing-set.csv')

In [3]:
# data

In [4]:
# print data['proto'].value_counts()
# print data['proto'].nunique()
# print data['service'].value_counts()
# print data['service'].nunique()
# print data['state'].value_counts()
# print data['state'].nunique()


In [5]:
print data['attack_cat'].value_counts()
print data['attack_cat'].nunique()

Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
Name: attack_cat, dtype: int64
10


In [6]:
def build_dict(values):
    ans = dict()
    for i, x in enumerate(values):
        ans[x] = i
    return ans

In [7]:
protos = build_dict(data.proto.unique())
services = build_dict(data.service.unique())
states = build_dict(data.state.unique())

In [8]:
data.iloc[1, -1]

0

In [9]:
data.proto.replace(to_replace=protos, inplace=True)
data.service.replace(to_replace=services, inplace=True)
data.state.replace(to_replace=states, inplace=True)

In [21]:
dataset = data.iloc[:, 1:-2].values.astype(np.float32)
labels = data.iloc[:, -1].values.astype(np.float32)

In [20]:
dataset.shape

(82332, 42)

In [22]:
labels.shape

(82332,)

In [23]:
train_size = 62332
test_size = 10000
valid_size = 10000

In [24]:
np.random.seed(133)
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

In [25]:
dataset, labels = randomize(dataset, labels)


In [29]:
train_dataset = dataset[:train_size, :]
train_labels = labels[:train_size]
valid_dataset = dataset[train_size:train_size + valid_size, :]
valid_labels = labels[train_size:train_size + valid_size]
test_dataset = dataset[train_size + valid_size :, :]
test_labels = labels[train_size + valid_size:]

In [30]:
print train_dataset.shape
print train_labels.shape
print valid_dataset.shape
print valid_labels.shape
print test_dataset.shape
print test_labels.shape

(62332, 42)
(62332,)
(10000, 42)
(10000,)
(10000, 42)
(10000,)


In [31]:
pickle_file = 'nusm.pickle'
try:
    f = open(pickle_file, 'wb')
    save = {
        'train_dataset': train_dataset,
        'train_labels': train_labels,
        'valid_dataset': valid_dataset,
        'valid_labels': valid_labels,
        'test_dataset': test_dataset,
        'test_labels': test_labels,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise